In [0]:
import json
from pprint import pprint
from google.colab import auth
from googleapiclient.discovery import build
import io
from googleapiclient.http import MediaIoBaseDownload
import pickle

In [0]:
auth.authenticate_user()
drive_service = build('drive', 'v3')

In [6]:
def _create_file_request(file_id):
    return drive_service.files().get_media(fileId=file_id)
 
 
def _download_response_bytes(request, print_progress=False):
    downloaded = io.BytesIO()
    downloader = MediaIoBaseDownload(downloaded, request)
   
    for status in _progbar(downloader):
        if print_progress:
            print("Downloaded {}/{} bytes".format(status.resumable_progress, status.total_size))
 
    downloaded.seek(0)
    return downloaded.read()
 
 
def _progbar(downloader):
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        yield status
 
 
def get_file_id(name):
    return get_matching_files(name)[0]['id']
 
 
def move_from_drive_to_disk(file_names, file_destinations):
    for file_name, dest in zip(file_names, file_destinations):
        file_id = get_file_id(file_name)
        print('Downloading file: "{}"'.format(file_name))
        file_bytes = _download_response_bytes(_create_file_request(file_id), print_progress=True)
        with open(dest, "wb") as f:
            f.write(file_bytes)
 
 
def load_pickled_files(file_names):
    for name in file_names:
        yield pickle.load(open(name, "rb"))
       
 
def get_matching_files(name):
    drive_files_response = _download_response_bytes(drive_service.files().list())
    drive_files_response_dict = json.loads(
        drive_files_response.decode('utf-8')
    )
    drive_files_dict = drive_files_response_dict['files']
 
    matching_records = [
        record
        for record in drive_files_dict
        if record['name'] == name
    ]
 
    no_records = len(matching_records)
    if no_records == 0:
        raise ValueError('no such file: "{}" on your Google Drive'.format(name))
    elif no_records > 1:
        print('warning: multiple matches for file "{}"'.format(name))
    return matching_records
 
 
example_file_name = ['ann_x.npy', 'ann_y.npy']
for i in example_file_name:
    example_file_id = get_file_id(i)
    move_from_drive_to_disk([i], [i])

ValueError: ignored

In [0]:
import os
os.listdir()

['.config', 'ann_y.npy', 'ann_x.npy', 'adc.json', 'sample_data']

In [0]:
import numpy as np
x = np.load('ann_x.npy')
y = np.load('ann_y.npy')

In [0]:
print(x.shape)
print(y.shape)

(15345, 784)
(15345,)


In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [0]:
classifier = Sequential()
classifier.add(Dense(1500, input_dim = 784, activation = 'relu'))
classifier.add(Dense(1000, activation = 'relu'))
classifier.add(Dense(500, activation = 'relu'))
classifier.add(Dense(1, activation = 'sigmoid'))
classifier.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1500)              1177500   
_________________________________________________________________
dense_6 (Dense)              (None, 1000)              1501000   
_________________________________________________________________
dense_7 (Dense)              (None, 500)               500500    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 501       
Total params: 3,179,501
Trainable params: 3,179,501
Non-trainable params: 0
_________________________________________________________________


In [0]:
classifier.fit(x,y, epochs = 20)

Epoch 1/20
15345/15345 [==============================] - 4s 287us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 2/20
15345/15345 [==============================] - 5s 297us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 3/20
15345/15345 [==============================] - 5s 296us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 4/20
15345/15345 [==============================] - 5s 301us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 5/20
15345/15345 [==============================] - 5s 298us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 6/20
15345/15345 [==============================] - 5s 296us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 7/20
15345/15345 [==============================] - 5s 297us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 8/20
15345/15345 [==============================] - 4s 280us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 9/20
15345/15345 [==============================] - 4s 279us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 10/20
15345/15345 [==============================

In [0]:
classifier.save_weights('weights_95.h5')

In [0]:
os.listdir()

['.config',
 'ann_y.npy',
 'weights_95.h5',
 'ann_x.npy',
 'adc.json',
 'sample_data']